<a href="https://colab.research.google.com/github/caroheymes/flask_app_colab/blob/master/first_flask_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My first flask app in colab

In [ ]:
!pip install flask-ngrok
!pip install flask-bootstrap

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/1_app_flask')

In [12]:
!pip install wget
import widgetsnbextension
!wget https://julie-2-next-resources.s3.eu-west-3.amazonaws.com/FULL_STACK_24_WEEK_PROGRAM/M09-Deployment/D06-Serve_your_model_with_API/build_a_spam_filter.zip

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=30fd5bcb8d734ac9d7f2df1d286ac626c9d904446f62db535c9d2f7f842f1052
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
--2021-11-03 22:31:14--  https://julie-2-next-resources.s3.eu-west-3.amazonaws.com/FULL_STACK_24_WEEK_PROGRAM/M09-Deployment/D06-Serve_your_model_with_API/build_a_spam_filter.zip
Resolving julie-2-next-resources.s3.eu-west-3.amazonaws.com (julie-2-next-resources.s3.eu-west-3.amazonaws.com)... 52.95.154.8
Connecting to julie-2-next-resources.s3.eu-west-3.amazonaws.com (julie-2-next-resources.s3.eu-west-3.amazonaws.com)|52.95.154.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2048364 (2.0M) [binary/octet-stream]
Saving to: ‘build_a_spam_filter.zip’

build_a_spam_filter 100%[===================>]   1.95M  3.03MB/s    in 0.6s    

2021-11-03 22:31:15 (3.03 MB/s) - ‘build_a_spam_filter.zip’ sa

In [15]:
!unzip build_a_spam_filter.zip

Archive:  build_a_spam_filter.zip
   creating: build_a_spam_filter/
 extracting: build_a_spam_filter/requirements.txt  
   creating: build_a_spam_filter/datas/
  inflating: build_a_spam_filter/datas/spam_dataset.csv  
   creating: build_a_spam_filter/datas/.ipynb_checkpoints/
  inflating: build_a_spam_filter/datas/deleteme.csv  
   creating: build_a_spam_filter/models/
  inflating: build_a_spam_filter/models/spam_classifier.joblib  
  inflating: build_a_spam_filter/train.py  
  inflating: build_a_spam_filter/app.py  


In [16]:
import os
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [18]:
df = pd.read_csv('build_a_spam_filter/datas/spam_dataset.csv')

In [22]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [25]:
ls

build_a_spam_filter/  build_a_spam_filter.zip  static/  templates/


In [26]:
# Get X and y
X = df["email"]
y = df["label"]
# Split dataset into train set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
# Build a pipeline with our CountVectorizer and our LogisticRegression model
classifier = Pipeline([("vectorizer", CountVectorizer()), ("classifier", LogisticRegression(solver="liblinear"))])
# Fit our classifier
# Fit our classifier
classifier.fit(X_train, y_train)
# Compute accuracy on test set
accuracy = accuracy_score(classifier.predict(X_test), y_test)
print("Accuracy: ", accuracy)
# Save our model with joblib
joblib.dump(classifier, os.path.join('/content/drive/MyDrive/1_app_flask/build_a_spam_filter/models', "spam_classifier.joblib"))

Accuracy:  0.992


['/content/drive/MyDrive/1_app_flask/build_a_spam_filter/models/spam_classifier.joblib']

In [50]:
from flask import Flask, url_for, request, jsonify,render_template
import joblib
import requests
from flask_ngrok import run_with_ngrok

TEMPLATE = './templates'
STATIC = './static'

app = Flask(__name__, 
            template_folder=TEMPLATE, 
            static_folder=STATIC)


run_with_ngrok(app)

@app.route("/")
def index():
   return render_template('index.html')
 
@app.route("/spam", methods=["POST"])
def pred():
    # return render_template('index.html')
    # Check if request has a JSON content
    if request.json:
        # Get the JSON as dictionnary
        req = request.get_json()
        # Check mandatory key
        if "email" in req.keys():
            # Load model
            classifier = joblib.load("./build_a_spam_filter/models/spam_classifier.joblib")
            # Predict
            prediction = classifier.predict([req["email"]])
            # Return the result as JSON but first we need to transform the
            # result so as to be serializable by jsonify()
            prediction = str(prediction[0])
            return jsonify({"predict": prediction}), 200
    return jsonify({"msg": "Error: not a JSON or no email key in your request"})


@app.route("/more_love")
def more_love():
    things_i_love = [
        "Star Wars",
        "Coffee",
        "Cookies",
    ]
    return render_template("more_love.html", things_i_love=things_i_love)


# # @app.route("/url_for_about")
# def urls():
#     return "{}".format(url_for("about"))

  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://dad3-35-231-242-231.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [03/Nov/2021 23:07:16] "POST /spam HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2021 23:08:27] "POST /spam HTTP/1.1" 200 -


In [ ]:
#Dans un autre notebook
import requests
response_non_spam = requests.post("http://dad3-35-231-242-231.ngrok.io/spam", json={"email": "money sex bitcoin his lawsuit against todd spawn mcfarlane vindicated in his assertion that mcfarlane breached his contracts stole his characters and used his name mcfarlane looked down somberly as the verdict was read as the judge polled the individual jury members he looked at their faces link number discuss number _thanks gnat number _ number url number url number url"})
response_non_spam.json()

In [51]:
#Pour éteindre l'app
app.do_teardown_appcontext()

# Autre version

In [ ]:
import os
import threading

from flask import Flask
from pyngrok import ngrok

# os.environ["FLASK_ENV"] = "development"

TEMPLATE = './templates'
STATIC = './static'


app = Flask(__name__,template_folder=TEMPLATE, 
            static_folder=STATIC)
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Define Flask routes
@app.route("/")
def index():
    return render_template('index.html')


@app.route("/more_love")
def more_love():
    things_i_love = [
        "Star Wars",
        "Coffee",
        "Cookies",
    ]
    return render_template("more_love.html", things_i_love=things_i_love)
# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel "http://2b79-35-231-242-231.ngrok.io" -> "http://127.0.0.1:5000"
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [ ]:
!ps -fA | grep python

root          41       7  0 17:20 ?        00:00:08 python3 /usr/local/bin/colab-fileshim.py
root          54       7  0 17:20 ?        00:00:19 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         878       1  0 18:44 ?        00:00:00 bash -c tail -n +0 -F "/root/.config/Google/DriveFS/Logs/drive_fs.txt" | python3 /opt/google/drive/drive-filter.py > "/root/.config/Google/DriveFS/Logs/timeouts.txt" 
root         880     878  0 18:44 ?        00:00:00 python3 /opt/google/drive/drive-filter.py
root        2531      54  0 20:24 ?        00:00:20 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-b9f229c3-34f6-4b53-92e7-610e55b862fb.json
root        2551       1  0 20:24 ?        00:00:08 /usr/bin/python3 /usr/local/lib/python3.7/dist-packages/debugpy/adapter --for-server 36033 --host 127.0.0.1 --port 15699 --server-access-token a5999fc9b34b

In [ ]:
!kill 3195

/bin/bash: line 0: kill: (3195) - No such process
